In [25]:
# Import necessary libraries for data handling, model operations, and visualization
import os
import json
import requests
import numpy as np
import torch
from torch import nn
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm

# Import specific tools from the transformers library
import transformers
from transformers import BertTokenizerFast, BertForQuestionAnswering, AdamW

In [2]:
# Define the base path for data storage on Palmetto
BASE_DIR = "/home/jrajend/HW3"

# Specify the filenames for the various dataset files
train_file = "spoken_train-v1.1.json"
test_file = "spoken_test-v1.1.json"
test_file_WER44 = "spoken_test-v1.1_WER44.json"
test_file_WER54 = "spoken_test-v1.1_WER54.json"

# Construct full paths
train_file_path = os.path.join(BASE_DIR, train_file)
test_file_path = os.path.join(BASE_DIR, test_file)
test_file_WER44_path = os.path.join(BASE_DIR, test_file_WER44)
test_file_WER54_path = os.path.join(BASE_DIR, test_file_WER54)


In [3]:
# Set the computing device based on GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Print device status for verification
if torch.cuda.is_available():
    print("Running on CUDA.")
else:
    print("CUDA not available; running on CPU.")

Running on CUDA.


In [4]:
import os

# Load the path for the training dataset file
train_file_path = os.path.join(BASE_DIR, train_file)
print("Training file path:", train_file_path)

# Repeat similar steps for other dataset files if required

Training file path: /home/jrajend/HW3/spoken_train-v1.1.json


In [5]:
# Function to extract contexts, questions, and answers from a JSON file
def load_data_from_json(path):
    # Initialize lists to store contexts, questions, and answers
    data_contexts, data_questions, data_answers = [], [], []

    # Open the JSON file and load its content
    with open(path, 'r') as file:
        file_content = json.load(file)

    # Process each section within the data file
    for entry in file_content.get('data', []):
        paragraphs = entry.get('paragraphs', [])

        # Extract context and question-answer pairs from each paragraph
        for paragraph in paragraphs:
            context_text = paragraph.get('context', "").lower()

            qas = paragraph.get('qas', [])
            for qa_pair in qas:
                question_text = qa_pair.get('question', "").lower()

                # Append each answer related to the question and context
                for answer in qa_pair.get('answers', []):
                    data_contexts.append(context_text)
                    data_questions.append(question_text)
                    data_answers.append(answer)

    # Print the first few entries to verify the output
    print("Sample Context:", data_contexts[:1])
    print("Sample Question:", data_questions[:1])
    print("Sample Answer:", data_answers[:1])

    return data_contexts, data_questions, data_answers

# Example usage with a file path
contexts, questions, answers = load_data_from_json(train_file_path)


Sample Context: ['architecturally the school has a catholic character. atop the main building school dome is the golden statue of the virgin mary. immediately in front of the main building in facing it is a copper statue of christ with arms appraised with the legend and the bad meow names. next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto im mary in place of prayer and reflection. it is a replica of the grotto at lourdes france where the virgin mary reputedly appeared to st bernadette still burning eighteen fifty eight. at the end of the main drive and in a direct line that connects through three statues in the gold dome is as simple modern stone statue of mary.']
Sample Question: ['what is in front of the notre dame main building?']
Sample Answer: [{'answer_start': 187, 'text': 'a copper statue of christ'}]


In [6]:
# Define paths for each dataset type
train_data_path = os.path.join(BASE_DIR, train_file)
test_data_path = os.path.join(BASE_DIR, test_file)
test_data_path_WER44 = os.path.join(BASE_DIR, test_file_WER44)
test_data_path_WER54 = os.path.join(BASE_DIR, test_file_WER54)

# Load and display training data sample
train_contexts, train_questions, train_answers = load_data_from_json(train_data_path)
print(f"Sample from Training Data:\nQuestion: {train_questions[0]}\nAnswer: {train_answers[0]}")

# Load and display testing data sample
test_contexts, test_questions, test_answers = load_data_from_json(test_data_path)
print(f"Sample from Testing Data:\nQuestion: {test_questions[0]}\nAnswer: {test_answers[0]}")

# Load and display WER 44 testing data sample
test_contexts_44, test_questions_44, test_answers_44 = load_data_from_json(test_data_path_WER44)
print(f"Sample from Testing Data WER 44:\nQuestion: {test_questions_44[0]}\nAnswer: {test_answers_44[0]}")

# Load and display WER 54 testing data sample
test_contexts_54, test_questions_54, test_answers_54 = load_data_from_json(test_data_path_WER54)
print(f"Sample from Testing Data WER 54:\nQuestion: {test_questions_54[0]}\nAnswer: {test_answers_54[0]}")

Sample Context: ['architecturally the school has a catholic character. atop the main building school dome is the golden statue of the virgin mary. immediately in front of the main building in facing it is a copper statue of christ with arms appraised with the legend and the bad meow names. next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto im mary in place of prayer and reflection. it is a replica of the grotto at lourdes france where the virgin mary reputedly appeared to st bernadette still burning eighteen fifty eight. at the end of the main drive and in a direct line that connects through three statues in the gold dome is as simple modern stone statue of mary.']
Sample Question: ['what is in front of the notre dame main building?']
Sample Answer: [{'answer_start': 187, 'text': 'a copper statue of christ'}]
Sample from Training Data:
Question: what is in front of the notre dame main building?
Answer: {'answer_start': 187, 'tex

In [7]:
# Function to add the 'answer_end' index for each answer in the dataset
def set_answer_end_indices(answers, contexts):
    for ans, ctx in zip(answers, contexts):
        ans_text = ans['text'].lower()
        ans_start = ans['answer_start']
        ans_end = ans_start + len(ans_text)

        # Check if the text matches at the expected location
        if ctx[ans_start:ans_end] == ans_text:
            ans['answer_end'] = ans_end
        else:
            # Adjust start and end indices if there is a mismatch
            for adjustment in [1, 2]:
                shifted_start = ans_start - adjustment
                shifted_end = ans_end - adjustment
                if ctx[shifted_start:shifted_end] == ans_text:
                    ans['answer_start'] = shifted_start
                    ans['answer_end'] = shifted_end
                    break  # Stop adjustment once a match is found

In [8]:
# Set answer end indices for each dataset by calling the function
set_answer_end_indices(train_answers, train_contexts)
set_answer_end_indices(test_answers, test_contexts)
set_answer_end_indices(test_answers_44, test_contexts_44)
set_answer_end_indices(test_answers_54, test_contexts_54)

In [9]:
# Define maximum context length for truncation
MAX_CONTEXT_LENGTH = 512
train_contexts_truncated = []

# Iterate through each context to apply truncation if needed
for idx in range(len(train_contexts)):
    current_context = train_contexts[idx]
    current_answer = train_answers[idx]

    # Check if the context length exceeds the maximum allowed length
    if len(current_context) > MAX_CONTEXT_LENGTH:
        ans_start = current_answer['answer_start']
        ans_end = ans_start + len(current_answer['text'])
        
        # Calculate the middle point around the answer span
        answer_midpoint = (ans_start + ans_end) // 2
        context_start = max(0, min(answer_midpoint - MAX_CONTEXT_LENGTH // 2, len(current_context) - MAX_CONTEXT_LENGTH))
        context_end = context_start + MAX_CONTEXT_LENGTH
        
        # Append the truncated context and adjust answer start index
        train_contexts_truncated.append(current_context[context_start:context_end])
        current_answer['answer_start'] = max(0, (MAX_CONTEXT_LENGTH // 2) - len(current_answer['text']))
    else:
        # If no truncation is needed, append the original context
        train_contexts_truncated.append(current_context)


In [13]:
# Set maximum length and model details for tokenization
MAX_CONTEXT_LENGTH = 512
MODEL_NAME = "bert-base-uncased"
DOC_STRIDE = 128

# Initialize the tokenizer for the specified BERT model
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

# Tokenize each dataset with the defined parameters
train_encodings = tokenizer(train_questions, train_contexts_truncated, max_length=MAX_CONTEXT_LENGTH, 
                            truncation=True, padding=True, stride=DOC_STRIDE)

# Tokenize the test data, referring to variables established in earlier modules
test_encodings = tokenizer(test_questions, test_contexts, max_length=MAX_CONTEXT_LENGTH, 
                           truncation=True, padding=True, stride=DOC_STRIDE)

# Tokenize WER 44 and WER 54 test data
test_encodings_44 = tokenizer(test_questions_44, test_contexts_44, max_length=MAX_CONTEXT_LENGTH, 
                              truncation=True, padding=True, stride=DOC_STRIDE)

test_encodings_54 = tokenizer(test_questions_54, test_contexts_54, max_length=MAX_CONTEXT_LENGTH, 
                              truncation=True, padding=True, stride=DOC_STRIDE)


In [14]:
# Function to locate start and end positions of answers within tokenized encodings
def locate_answer_positions(encodings, answers, tokenizer):
    start_positions, end_positions = [], []

    # Iterate over each encoding-answer pair
    for idx in range(len(encodings['input_ids'])):
        answer_text = answers[idx]['text']
        
        # Tokenize the answer text independently
        answer_tokens = tokenizer(answer_text, max_length=MAX_CONTEXT_LENGTH, truncation=True, padding=True)

        # Initialize position tracking variables
        answer_start, answer_end = 0, 0
        answer_found = False

        # Search for matching token sequence within the context
        context_tokens = encodings['input_ids'][idx]
        for j in range(len(context_tokens) - len(answer_tokens['input_ids'])):
            if context_tokens[j + 1:j + len(answer_tokens['input_ids']) - 1] == answer_tokens['input_ids'][1:-1]:
                answer_start = j
                answer_end = j + len(answer_tokens['input_ids']) - 1
                answer_found = True
                break

        # Append positions or default values if no match was found
        start_positions.append(answer_start if answer_found else 0)
        end_positions.append(answer_end if answer_found else 0)

    return start_positions, end_positions

# Generate and add start/end positions for each dataset encoding
# Ensure this block of code is run after tokenizing each dataset

# For training data
train_start_positions, train_end_positions = locate_answer_positions(train_encodings, train_answers, tokenizer)
train_encodings.update({'start_positions': train_start_positions, 'end_positions': train_end_positions})

# For test data
test_start_positions, test_end_positions = locate_answer_positions(test_encodings, test_answers, tokenizer)
test_encodings.update({'start_positions': test_start_positions, 'end_positions': test_end_positions})

# For WER 44 test data
test_start_positions_44, test_end_positions_44 = locate_answer_positions(test_encodings_44, test_answers_44, tokenizer)
test_encodings_44.update({'start_positions': test_start_positions_44, 'end_positions': test_end_positions_44})

# For WER 54 test data
test_start_positions_54, test_end_positions_54 = locate_answer_positions(test_encodings_54, test_answers_54, tokenizer)
test_encodings_54.update({'start_positions': test_start_positions_54, 'end_positions': test_end_positions_54})


In [15]:
class QADataset(Dataset):
    def __init__(self, encodings):
        # Store all encoding tensors in a dictionary for efficient access
        self.data = {
            'input_ids': torch.tensor(encodings['input_ids']),
            'token_type_ids': torch.tensor(encodings['token_type_ids']),
            'attention_mask': torch.tensor(encodings['attention_mask']),
            'start_positions': torch.tensor(encodings['start_positions']),
            'end_positions': torch.tensor(encodings['end_positions'])
        }

    def __getitem__(self, idx):
        # Retrieve each tensor directly from the data dictionary
        return {key: value[idx] for key, value in self.data.items()}

    def __len__(self):
        # Return the total number of samples based on input IDs
        return len(self.data['input_ids'])

In [17]:
# Create dataset instances for each set of encodings
train_dataset = QADataset(train_encodings)
test_dataset = QADataset(test_encodings)
test_dataset_44 = QADataset(test_encodings_44)
test_dataset_54 = QADataset(test_encodings_54)


In [18]:
from transformers import BertForQuestionAnswering

# Initialize and load the pre-trained BERT model for question answering
model_name = "bert-base-uncased"
qa_model = BertForQuestionAnswering.from_pretrained(model_name)

# Confirm successful loading by printing the model architecture
print("Loaded BERT model for QA:", qa_model)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded BERT model for QA: BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

In [20]:
from transformers import get_scheduler

# Function to train the model with specified data loader and number of epochs
def train_qa_model(qa_model, train_loader, epochs=1):
    qa_model.to(device)
    qa_model.train()

    # Initialize optimizer and learning rate scheduler
    optimizer = AdamW(qa_model.parameters(), lr=2e-5, weight_decay=2e-2)
    total_training_steps = epochs * len(train_loader)
    scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=total_training_steps,
    )

    epoch_losses = []
    epoch_accuracies = []

    for epoch in range(epochs):
        batch_losses = []
        batch_accuracies = []
        with tqdm(train_loader, desc=f'Epoch {epoch + 1}') as progress_bar:

            for batch in progress_bar:
                optimizer.zero_grad()

                # Transfer data to the correct device
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                token_type_ids = batch['token_type_ids'].to(device)
                start_positions = batch['start_positions'].to(device)
                end_positions = batch['end_positions'].to(device)

                # Forward pass
                outputs = qa_model(input_ids, attention_mask=attention_mask,
                                   start_positions=start_positions,
                                   token_type_ids=token_type_ids,
                                   end_positions=end_positions)

                loss = outputs[0]
                loss.backward()
                optimizer.step()
                scheduler.step()

                # Calculate accuracy for start and end predictions
                start_logits, end_logits = outputs[1], outputs[2]
                start_preds = start_logits.argmax(dim=1)
                end_preds = end_logits.argmax(dim=1)
                accuracy = ((start_preds == start_positions).float().mean() +
                            (end_preds == end_positions).float().mean()) / 2

                batch_losses.append(loss.item())
                batch_accuracies.append(accuracy.item())

                # Display current loss and accuracy in progress bar
                progress_bar.set_postfix_str(f'Loss: {loss.item():.4f}, Accuracy: {accuracy.item():.4f}')

        epoch_losses.append(np.mean(batch_losses))
        epoch_accuracies.append(np.mean(batch_accuracies))

    return epoch_losses, epoch_accuracies


In [22]:
from torch.utils.data import DataLoader

# Set batch size for training
BATCH_SIZE = 16

# Initialize DataLoader for training data with defined batch size and shuffling
train_data_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Train the question-answering model and capture loss and accuracy history
train_loss_history, train_accuracy_history = train_qa_model(qa_model, train_data_loader, epochs=2)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/software/slurm/spackages/linux-rocky8-x86_64/gcc-12.2.0/anaconda3-2023.09-0-3mhml42fa64byxqyd5fig5tbih625dp2/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 2: 100%|██████████| 2320/2320 [01:49<00:00, 21.19it/s, Loss: 1.3694, Accuracy: 0.5714]


In [23]:
from collections import Counter

# Function to calculate the F1 score for a list of predicted and reference answers
def compute_f1_score(predictions, references):
    f1_scores = []
    
    # Iterate over each predicted and reference answer pair
    for prediction, reference in zip(predictions, references):
        # Count overlapping words between prediction and reference
        overlap = Counter(prediction) & Counter(reference)
        overlap_count = sum(overlap.values())
        
        # Handle cases with no overlap
        if overlap_count == 0:
            f1_scores.append(0)
            continue
        
        # Calculate precision and recall
        precision = overlap_count / len(prediction)
        recall = overlap_count / len(reference)
        
        # Compute F1 score and append to list
        f1 = (2 * precision * recall) / (precision + recall)
        f1_scores.append(f1)
    
    # Calculate and return the average F1 score
    average_f1_score = sum(f1_scores) / len(f1_scores)
    return average_f1_score


In [24]:
from torch.utils.data import DataLoader

# Set parameters for evaluation
N_BEST = 20
MAX_ANSWER_LENGTH = 30

# Function to evaluate the model using the provided data loader
def evaluate_qa_model(qa_model, eval_loader):
    qa_model.eval()
    predictions = []
    references = []

    with torch.no_grad():
        for batch in eval_loader:
            # Move input tensors to the specified device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            start_positions = batch['start_positions'].to(device)
            end_positions = batch['end_positions'].to(device)

            # Forward pass to get model outputs
            outputs = qa_model(input_ids, attention_mask=attention_mask,
                               start_positions=start_positions,
                               token_type_ids=token_type_ids,
                               end_positions=end_positions)

            start_logits, end_logits = outputs.start_logits, outputs.end_logits
            start_preds = torch.argmax(start_logits, dim=1)
            end_preds = torch.argmax(end_logits, dim=1)

            # Decode predicted and reference answers
            for i in range(len(start_preds)):
                start_idx = start_preds[i].item()
                end_idx = end_preds[i].item()

                # Skip answers where start index > end index or length exceeds max answer length
                if start_idx > end_idx or end_idx - start_idx + 1 > MAX_ANSWER_LENGTH:
                    predicted_answer = ""
                else:
                    predicted_answer = tokenizer.decode(batch['input_ids'][i][start_idx:end_idx + 1], skip_special_tokens=True)

                predictions.append(predicted_answer)

                # Decode reference answer
                ref_start = batch['start_positions'][i].item()
                ref_end = batch['end_positions'][i].item()
                reference_answer = tokenizer.decode(batch['input_ids'][i][ref_start:ref_end + 1], skip_special_tokens=True)
                references.append(reference_answer)

    # Calculate average F1 score over all predictions
    avg_f1_score = compute_f1_score(predictions, references)
    return avg_f1_score

# Data loaders for each test dataset
test_data_loader = DataLoader(test_dataset, batch_size=16)
test_data_loader_44 = DataLoader(test_dataset_44, batch_size=16)
test_data_loader_54 = DataLoader(test_dataset_54, batch_size=16)

# Evaluate model on each dataset and print F1 scores
f1_score = evaluate_qa_model(qa_model, test_data_loader)
print(f"F1 Score on Test Data: {f1_score}")

f1_score_44 = evaluate_qa_model(qa_model, test_data_loader_44)
print(f"F1 Score on Test Data WER 44: {f1_score_44}")

f1_score_54 = evaluate_qa_model(qa_model, test_data_loader_54)
print(f"F1 Score on Test Data WER 54: {f1_score_54}")


F1 Score on Test Data: 0.7179008439005641
F1 Score on Test Data WER 44: 0.4049083617023346
F1 Score on Test Data WER 54: 0.2880895915805823
